# <u>S</u>tabilizer <u>S</u>ub<u>S</u>pace (S3) Projections

A library providing the necessary functionality to perform stabilizer subspace projections over systems of Pauli 
operators.

This facilitates implementations of qubit reduction techniques such as [*tapering*](https://arxiv.org/abs/1701.08213) and [*Contextual-Subspace VQE*](https://doi.org/10.22331/q-2021-05-14-456).

In [1]:
from qreduce.cs_vqe import cs_vqe
from qreduce.utils.operator_toolkit import *
from itertools import combinations
import json
import numpy as np

Will store matrices in sparse form


/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
import ast
f = open('data/hamiltonians.txt', 'r')
hamiltonians = ast.literal_eval(f.read())
f.close()

print(hamiltonians.keys())

dict_keys(['H2-S1_STO-3G_singlet', 'C1-O1_STO-3G_singlet', 'H1-Cl1_STO-3G_singlet', 'H1-Na1_STO-3G_singlet', 'H2-Mg1_STO-3G_singlet', 'H1-F1_3-21G_singlet', 'H1-Li1_3-21G_singlet', 'Be1_STO-3G_singlet', 'H1-F1_STO-3G_singlet', 'H1-Li1_STO-3G_singlet', 'Ar1_STO-3G_singlet', 'F2_STO-3G_singlet', 'H1-O1_STO-3G_singlet', 'H2-Be1_STO-3G_singlet', 'H2-O1_STO-3G_singlet', 'H2_3-21G_singlet', 'H2_6-31G_singlet', 'H3-N1_STO-3G_singlet', 'H4-C1_STO-3G_singlet', 'Mg1_STO-3G_singlet', 'N2_STO-3G_singlet', 'Ne1_STO-3G_singlet', 'O2_STO-3G_singlet', 'H1-Li1-O1_STO-3G_singlet', 'H1-He1_STO-3G_singlet', 'H3_STO-3G_singlet_1+', 'H1-He1_3-21G_singlet_1+', 'H3_3-21G_singlet_1+', 'H4-N1_STO-3G_singlet_1+'])


In [3]:
speciesname = 'H2-O1_STO-3G_singlet'#'Be1_STO-3G_singlet'##

encoding = hamiltonians[speciesname][0] # in this dataset, always 'JW' for Jordan-Wigner, but leaves room for trying Bravyi-Kitaev as well
n_qubits = hamiltonians[speciesname][1] # number of qubits (all of these Hamiltonians have been tapered for molecular symmetries)
ham = hamiltonians[speciesname][2] # full Hamiltonian
ham_noncon = hamiltonians[speciesname][3] # noncontextual part of Hamiltonian, found by greedy DFS
exact = hamiltonians[speciesname][4] # ground state energy of full Hamiltonian (in Hartree)
gs_noncon = hamiltonians[speciesname][5] # list containing information about noncontextual ground state: zeroth entry is ground state energy of noncontextual part of Hamiltonian


In [4]:
from qreduce.utils.cs_vqe_tools_legacy import csvqe_approximations_heuristic, quasi_model

csvqe = csvqe_approximations_heuristic(ham, 
                                       ham_noncon, 
                                       n_qubits, 
                                       exact)

print('true ground state energy:', csvqe[0], '\n')
print('CS-VQE approximations:', csvqe[1], '\n')
print('CS-VQE errors:', csvqe[2], '\n')
print('chosen order:', csvqe[3])

true ground state energy: -83.92870248174707 

CS-VQE approximations: [-83.87422390061542, -83.87422390061542, -83.87959838666475, -83.8831584734779, -83.89785356786001, -83.91205889216403, -83.91813131301437, -83.92761703951186, -83.92862837750941, -83.92865354102875, -83.92870248174692] 

CS-VQE errors: [0.054478581131647275, 0.054478581131647275, 0.04910409508231339, 0.04554400826916094, 0.030848913887055573, 0.016643589583040352, 0.010571168732695924, 0.0010854422352082338, 7.410423765463747e-05, 4.894071831529345e-05, 1.4210854715202004e-13] 

chosen order: [2, 9, 4, 0, 3, 6, 5, 1, 8, 7]


# Qubit Tapering

Here we run through the basic tapering functionality

In [5]:
from qreduce.tapering import tapering
print(ham)
tapering(ham).find_symmetry()

{'IIIIIIIIII': -55.226577416214326, 'IIIIIIIIIZ': 12.411397933347452, 'IIIIIIIIYY': -0.007573527729508396, 'IIIIIIIIZI': 12.411397933347452, 'IIIIIIIIZZ': 1.1862777224861802, 'IIIIIIIXIX': -0.10460420805448783, 'IIIIIIIXXI': 0.0020603107584759067, 'IIIIIIIXZX': -0.12499935118669338, 'IIIIIIIYIY': -0.10460420805448783, 'IIIIIIIYZY': -0.12499935118669338, 'IIIIIIIZII': 1.6496131029811243, 'IIIIIIIZIZ': 0.23693603049649825, 'IIIIIIIZZI': 0.2515869355056082, 'IIIIIIXIXI': -0.0033258799919286815, 'IIIIIIXXYY': -0.014650905009109934, 'IIIIIIXYYX': 0.014650905009109934, 'IIIIIIXZXI': -0.12499935118669338, 'IIIIIIXZXZ': -0.10460420805448783, 'IIIIIIYIYI': -0.0033258799919286815, 'IIIIIIYXXY': 0.014650905009109934, 'IIIIIIYYII': -0.025296526569690943, 'IIIIIIYYXX': -0.014650905009109934, 'IIIIIIYZYI': -0.12499935118669338, 'IIIIIIYZYZ': -0.10460420805448783, 'IIIIIIYZZY': 0.0020603107584759067, 'IIIIIIZIII': 1.6496131029811243, 'IIIIIIZIIZ': 0.2515869355056082, 'IIIIIIZIZI': 0.23693603049649825

# Contextual-Subspace VQE
Here we run through the basic CS-VQE functionality...

When the `cs_vqe` class is initiated it generates a set of stabilizers defined through the CS-VQE method. These stabilizers are consistent with the noncontextual ground state energy, a classical estimate of the true value that is *at least as accurate* as Hartree-Fock.

In [6]:
cs = cs_vqe(ham, noncontextual_set = list(ham_noncon.keys()), single_pauli='Z')

match_original = abs(cs.ngs_energy-gs_noncon[0])<1e-13
print("Noncontextual GS energy:",  cs.ngs_energy, ' // matches original?', match_original)

print("Symmetry generators:    ", cs.generators)
print("Clique representatives: ", cs.cliquereps)
print("Generator eigenvalues:  ", cs.nu)
print("Clique operator coeffs: ", cs.r)

Noncontextual GS energy: -83.8742239006154  // matches original? True
Symmetry generators:     ['IIIIIZIIII', 'ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII', 'IIIIZIIIII', 'IIIIIIZIII', 'IIIIIIIZII', 'IIIIIIIIZI', 'IIIIIIIIIZ']
Clique representatives:  ['ZIIIIXIZZZ', 'IIIIIZIIII']
Generator eigenvalues:   [ 1  1 -1 -1 -1 -1 -1 -1 -1 -1]
Clique operator coeffs:  [ 2.13055929e-07 -1.00000000e+00]


In [7]:
all([G in cs.symmetry for G in cs.generators[1:]])

True

In [8]:
chemaccnumq = list(np.array(csvqe[2])<0.0016).index(True) #mol['chem_acc_num_q']

#exact, gs = exact_gs_energy(cs.hamiltonian)
print('Exact energy:',exact)
print('Noncon error:', cs.ngs_energy-exact)
print(f'Target_error for {chemaccnumq} qubits:', csvqe[2][chemaccnumq])# exact_gs_energy(mol['ham_reduced'][num_sim_q])[0]-exact)

Exact energy: -83.92870248174707
Noncon error: 0.054478581131661485
Target_error for 7 qubits: 0.0010854422352082338


**CS-VQE is sensitive to the choice of stabilizers we wish to enforce.**

Below, we drop stabilizer constraints iteratively, choosing that which minimizes the energy at each step.

In [9]:
stab_index_pool = list(range(len(cs.generators)))

optimal_errors = {}
for num_sim_q in [chemaccnumq]: #range(1,cs.n_qubits):
    cs_vqe_errors = []
    for order in combinations(range(len(cs.generators)), cs.n_qubits - num_sim_q):
        order = list(order)
        ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=order)#list(range(cs_vqe_mol.num_qubits)),
                                                                    #projection_qubits=order)
        cs_energy, cs_vector = exact_gs_energy(ham_cs)
        cs_vqe_errors.append((cs_energy-exact, order))
        
    cs_vqe_errors = sorted(cs_vqe_errors, key=lambda x:x[0])
    error, stab_index_pool = cs_vqe_errors[0]
    
    optimal_errors[num_sim_q]={}
    optimal_errors[num_sim_q]['error'] = error
    optimal_errors[num_sim_q]['stab_indices'] = list(stab_index_pool)
    
for num_sim_q in optimal_errors:
    error = optimal_errors[num_sim_q]['error']
    diff_will = error-csvqe[2][num_sim_q]
    print(diff_will)
    stab_indices = optimal_errors[num_sim_q]['stab_indices']
    print(f'Performing {num_sim_q}-qubit CS-VQE, we may obtain',
          f'an absolute error of {error:.8f},\n',
          f'enforcing the stabilizers {stab_indices}, {[cs.generators[i] for i in stab_indices]}\n'
         )

-2.931949438789161e-08
Performing 7-qubit CS-VQE, we may obtain an absolute error of 0.00108541,
 enforcing the stabilizers [2, 8, 9], ['IZIIIIIIII', 'IIIIIIIIZI', 'IIIIIIIIIZ']



Suppose we have access to just 3 qubits on some quantum device... then we may construct the corresponding 3-qubit CS-VQE model, obtaining the reduced Hamiltonian, Ansatz operator and noncontextual reference state.

In [10]:
num_sim_q = 3
stab_indices = optimal_errors[num_sim_q]['stab_indices']
ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=stab_indices)
anz_cs = cs._contextual_subspace_projection(operator=ansatz,stabilizer_indices=stab_indices)
ngs = cs.noncontextual_ground_state(stabilizer_indices=stab_indices)

KeyError: 3

In [ ]:
print('Reduced Hamiltonian:\n', ham_cs)
print('\nReduced Ansatz:\n', anz_cs)
print('\nReference state:', ngs)

In [ ]:
plot_ground_state_amplitudes(operator=ham_cs, num_qubits=num_sim_q)#, reverse_bitstrings=True)

## Running CS-VQE

Finally, we may perform a VQE routine taking as input the reduced Hamiltonian and Ansatz defined above.

In [ ]:
import warnings; warnings.filterwarnings("ignore", category=DeprecationWarning)
import qreduce.utils.circuit_tools as circ
import qreduce.utils.circuit_execution_tools as circ_ex
from qiskit.circuit import QuantumCircuit

In [ ]:
qc = QuantumCircuit(num_sim_q)
for index, bit in enumerate(ngs):
    q_pos = num_sim_q-1-index
    if int(bit)==1:
        qc.x(q_pos)

qc = circ.circ_from_paulis(circ=qc, paulis=list(anz_cs.keys()), trot_order=2)
circ.cancel_pairs(circ=qc, hit_set={'s', 'sdg'})
circ.cancel_pairs(circ=qc, hit_set={'h', 'h'})
bounds = np.array([(a-np.pi/2, a+np.pi/2) for a in anz_cs.values()]) # optimization bounds
qc.parameter_bounds = bounds

qc.draw(output='mpl')

In [ ]:
vqe_result = circ_ex.vqe_simulation(ansatz=qc, operator=ham_cs, init_params=np.array(list(anz_cs.values())))

In [ ]:
log_errors = np.log10(np.square(np.array(vqe_result['values'])-exact))
plt.plot(log_errors, color='black', label='Optimizer output')
plt.hlines(np.log10(0.0016**2), 0, vqe_result['counts'][-1], label='Chemical accuracy', color='green')
plt.legend()

(-83.92870248174692,
 array([-5.08225092e-17+3.72994373e-18j, -3.10014048e-18+6.90805446e-18j,
        -8.59649869e-18-3.64130921e-18j, ...,
         2.69488002e-17-3.69663448e-17j, -3.88947949e-17-2.60789948e-17j,
        -2.88629757e-16-1.44063355e-15j]))